<a href="https://colab.research.google.com/github/Fifs251/thesis-experiment/blob/main/TF_training_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os
import time

In [ ]:
print(tf.__version__)

2.9.2


In [ ]:
(train_ds, validation_ds, test_ds), ds_info = tfds.load(
    "cifar100",
    split=["train[5000:]", "train[2500:5000]", "train[:2500]"],
    as_supervised=True,
    with_info=True,
)

In [ ]:
def process_images(image, label):
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image, (227,227))
    return image, label

In [ ]:
def augment(image, label):
  if tf.random.uniform((), minval=0, maxval=1) < 0.1:
    image =  tf.tile(tf.image.rgb_to_grayscale(image), [1,1,3])

  image = tf.image.random_brightness(image, max_delta=0.1)
  image = tf.image.random_contrast(image, lower=0.1, upper=0.2)
  image = tf.image.random_flip_left_right(image)

  return image, label

In [ ]:
def preproc(data):
  data=(data
          .map(process_images, num_parallel_calls=tf.data.AUTOTUNE)
          .shuffle(buffer_size=tf.data.experimental.cardinality(data).numpy())
          .batch(batch_size=32, drop_remainder=True)
          #.cache()
          #.prefetch(tf.data.AUTOTUNE)
          )
  return data

train_ds = preproc(train_ds)
test_ds = preproc(test_ds)
validation_ds = preproc(validation_ds)

In [ ]:
def random_crop(image):
  cropped_image = tf.image.random_crop(
      image, size=[224, 224, 3])

  return cropped_image

In [ ]:
from tensorflow.keras import models, layers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Activation, Dropout, BatchNormalization

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomBrightness(0.2),
  tf.keras.layers.RandomCrop(224, 224)
])

In [ ]:
def get_model():
  model = keras.models.Sequential([
    data_augmentation,
    Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPool2D(pool_size=(3,3), strides=(2,2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='softmax')
  ])

  return model

In [ ]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")

def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
    
run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(log_dir=run_logdir)

In [ ]:
model = get_model()

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.01), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
model.fit(train_ds,
          epochs=80,
          validation_data=validation_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])

Epoch 1/80


In [ ]:
model.evaluate(test_ds)

In [ ]:
model.summary()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs